In [1]:
import numpy
import sympy
from sympy import *
from sympy.matrices import *
from sympy.printing import ccode

from generate_cpp_code import *
from utils import *

## Point-Point

### 2D

In [2]:
p0 = numpy.array(sympy.symbols("p0_x p0_y"))
p1 = numpy.array(sympy.symbols("p1_x p1_y"))
x = numpy.concatenate([p0.T, p1.T])

In [3]:
p0_to_p1 = normalize(p1 - p0)

T = numpy.array([-p0_to_p1[1], p0_to_p1[0]]).reshape(-1, 1)

In [4]:
generators = [CXXJacobianGenerator(T, x, "point_point_tangent_basis_2D_jacobian")]

### 3D

In [5]:
p0 = numpy.array(sympy.symbols("p0_x p0_y p0_z"))
p1 = numpy.array(sympy.symbols("p1_x p1_y p1_z"))
x = numpy.concatenate([p0.T, p1.T])

In [6]:
T = numpy.empty([3, 2], dtype=object)

p0_to_p1 = p1 - p0

cross_x = numpy.cross(numpy.array([1, 0, 0]), p0_to_p1)
cross_y = numpy.cross(numpy.array([0, 1, 0]), p0_to_p1)

cross = numpy.array([
    sympy.Piecewise(
        (cross_x[i], sq_norm(cross_x) > sq_norm(cross_y)), 
        (cross_y[i], True))
    for i in range(3)])

T[:,0] = normalize(cross)
T[:,1] = normalize(numpy.cross(p0_to_p1, cross))

sympy.Matrix(T).diff(x).shape

(6, 3, 2)

In [7]:
generators.append(CXXJacobianGenerator(T, x, "point_point_tangent_basis_3D_jacobian"))

## Point-Edge

### 2D

In [8]:
p = numpy.array(sympy.symbols("p_x p_y"))
e0 = numpy.array(sympy.symbols("e0_x e0_y"))
e1 = numpy.array(sympy.symbols("e1_x e1_y"))
x = numpy.concatenate([p.T, e0.T, e1.T])

In [9]:
T = numpy.empty([2, 1], dtype=object)

T[:, 0] = normalize(e1 - e0)

generators.append(CXXJacobianGenerator(T, x, "point_edge_tangent_basis_2D_jacobian"))

### 3D

In [10]:
p = numpy.array(sympy.symbols("p_x p_y p_z"))
e0 = numpy.array(sympy.symbols("e0_x e0_y e0_z"))
e1 = numpy.array(sympy.symbols("e1_x e1_y e1_z"))
x = numpy.concatenate([p.T, e0.T, e1.T])

In [11]:
T = numpy.empty([3, 2], dtype=object)

e = e1 - e0
T[:, 0] = normalize(e)
T[:, 1] = normalize(numpy.cross(e, p - e0))

generators.append(CXXJacobianGenerator(T, x, "point_edge_tangent_basis_3D_jacobian"))

## Edge-Edge

In [12]:
ea0 = numpy.array(sympy.symbols("ea0_x ea0_y ea0_z"))
ea1 = numpy.array(sympy.symbols("ea1_x ea1_y ea1_z"))
eb0 = numpy.array(sympy.symbols("eb0_x eb0_y eb0_z"))
eb1 = numpy.array(sympy.symbols("eb1_x eb1_y eb1_z"))
x = numpy.concatenate([ea0.T, ea1.T, eb0.T, eb1.T])

In [13]:
T = numpy.empty([3, 2], dtype=object)

ea = ea1 - ea0
T[:, 0] = normalize(ea)

normal = numpy.cross(ea, eb1 - eb0);
T[:, 1] = normalize(numpy.cross(normal, ea));

generators.append(CXXJacobianGenerator(T, x, "edge_edge_tangent_basis_jacobian"))

## Point-Triangle

In [14]:
p = numpy.array(sympy.symbols("p_x p_y p_z"))
t0 = numpy.array(sympy.symbols("t0_x t0_y t0_z"))
t1 = numpy.array(sympy.symbols("t1_x t1_y t1_z"))
t2 = numpy.array(sympy.symbols("t2_x t2_y t2_z"))
x = numpy.concatenate([p.T, t0.T, t1.T, t2.T])

In [15]:
e0 = t1 - t0
normal = numpy.cross(e0, t2 - t0)
T = numpy.array([normalize(e0), normalize(numpy.cross(normal, e0))]).T

In [16]:
generators.append(CXXJacobianGenerator(T, x, "point_triangle_tangent_basis_jacobian"))

In [17]:
generate_hpp_file(generators, "tangent_basis_grad.hpp")
generate_cpp_file(generators, "tangent_basis_grad.cpp")